In [1]:
# Importing the libraries
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

2023-07-02 14:22:48.869585: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 14:22:48.927912: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-02 14:22:48.928907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 14:22:50.136071: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
print(tf.__version__)

2.12.0


### Preprocessing the training set

In [3]:
#applying transformations only in training set to avoid overfitting
#we will do some simple geometrical transformations as zoom, rotations
#(geometrical transformations like transvections) to shift some pixels
#then rotation, horizontal flips,some zoom in and zoom out to modify images

#technical term is image augmentation, so the model does not overlearn

#here we will rescale, shear, zoom and flip
#1./255 apply feature scaling between 0 and 1
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
#training_set in sinple quotes is the folder with subfolders of dog and cats
#target_size is the final size of images
training_set = train_datagen.flow_from_directory('training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing test set

In [4]:
#here we will not apply all the same transformation, we dont wanna touch the
#test images, however we need to reescale pixel
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


### Building the CNN

In [5]:
# Initialising the CNN
cnn = tf.keras.models.Sequential()

In [6]:
# Step 1 - Convolution

#conv2d --> filters is the number of feature detectors to apply
# input_shape is the input shape of your inputs, and colored pictures has 3D (RGB)
#for B/W images the last number would be 1
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [7]:
# Step 2 - Pooling
##maxpool is the max value of that region
##poolsize is the matrix to pool
#strides the window to move
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [8]:
# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [9]:
# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

In [10]:
# Step 4 - Full Connection
#as computer vision is more complex that data mining, we are gonna
#choose a larger number of hidden neurons
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [11]:
# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### training the CNN

In [12]:
#compiling the CNN

#adam is the stochastic gradient descent
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [13]:
# Training the CNN on the Training set and evaluating it on the Test set (instead of two steps)
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25


2023-07-02 14:22:52.729989: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - ETA: 0s - loss: 0.6610 - accuracy: 0.5999

2023-07-02 14:23:14.918010: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


250/250 [==============================] - 25s 96ms/step - loss: 0.6610 - accuracy: 0.5999 - val_loss: 0.6191 - val_accuracy: 0.6635
Epoch 2/25
250/250 [==============================] - 22s 89ms/step - loss: 0.5904 - accuracy: 0.6812 - val_loss: 0.5608 - val_accuracy: 0.7160
Epoch 3/25
250/250 [==============================] - 22s 89ms/step - loss: 0.5552 - accuracy: 0.7110 - val_loss: 0.5863 - val_accuracy: 0.6915
Epoch 4/25
250/250 [==============================] - 22s 88ms/step - loss: 0.5152 - accuracy: 0.7444 - val_loss: 0.5249 - val_accuracy: 0.7390
Epoch 5/25
250/250 [==============================] - 22s 88ms/step - loss: 0.4930 - accuracy: 0.7590 - val_loss: 0.5869 - val_accuracy: 0.7090
Epoch 6/25
250/250 [==============================] - 22s 90ms/step - loss: 0.4758 - accuracy: 0.7731 - val_loss: 0.5329 - val_accuracy: 0.7285
Epoch 7/25
250/250 [==============================] - 22s 89ms/step - loss: 0.4652 - accuracy: 0.7790 - val_loss: 0.4841 - val_accuracy: 0.7760
Epo

In [14]:
# Part 4 - Making a single prediction

import numpy as np
#from keras.preprocessing import image
#this one change by the next one
import keras.utils as image

test_image = image.load_img('single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
print(prediction)
print(result)

1/1 [==============================] - 0s 72ms/step
dog
[[1.]]


In [15]:
print(test_image)

[[[[239. 239. 239.]
   [239. 239. 239.]
   [239. 239. 239.]
   ...
   [245. 245. 245.]
   [245. 245. 245.]
   [245. 245. 245.]]

  [[239. 239. 239.]
   [239. 239. 239.]
   [239. 239. 239.]
   ...
   [245. 245. 245.]
   [245. 245. 245.]
   [244. 244. 244.]]

  [[239. 239. 239.]
   [239. 239. 239.]
   [239. 239. 239.]
   ...
   [244. 244. 244.]
   [244. 244. 244.]
   [244. 244. 244.]]

  ...

  [[241. 217. 205.]
   [243. 220. 206.]
   [242. 219. 205.]
   ...
   [226. 201. 181.]
   [233. 205. 193.]
   [232. 215. 197.]]

  [[255. 236. 229.]
   [251. 234. 227.]
   [252. 235. 227.]
   ...
   [219. 193. 178.]
   [219. 193. 178.]
   [216. 189. 172.]]

  [[243. 230. 224.]
   [250. 235. 228.]
   [245. 232. 226.]
   ...
   [203. 180. 164.]
   [220. 198. 177.]
   [225. 202. 186.]]]]
